# Probability Calibration in KG Embedding
This experiemnt is to investigate which calibration technique is the most suitable one given a dataset and a KG Embedding model.

Within this experiment, we are going to see the performance of 4 typical calibration techniques for 4 KGE models in 3 datasets:
- calibration techniques:
  - Platt Scaling
  - Isotonic Regression
  - Histogram BInning
  - Beta Calibration
- KG Embedding models
  - TransE
  - ComplEx
  - DistMult
  - HoLE
- Datasets
  - FB13k
  - Win11
  - Yago39

In [1]:
import sys
# enable importing the modules from probcalkge
sys.path.append('../')
sys.path.append('../probcalkge')

In [2]:
import importlib
from pprint import pprint
import numpy as np
import pandas as pd

In [3]:
from ampligraph.latent_features import RandomBaseline, TransE
import probcalkge
importlib.reload(probcalkge)
from probcalkge import Experiment
from probcalkge import get_calibrators
from probcalkge import get_datasets, get_fb13, get_wn11, get_kgemodels, get_yago39
from probcalkge import brier_score, negative_log_loss, ks_error

In [4]:
ds = get_datasets()
cals = get_calibrators()
kges = get_kgemodels()


WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


In [5]:
exp = Experiment(
    cals=[cals.uncal, cals.platt, cals.isot, cals.histbin, cals.beta, cals.temperature], 
    datasets=[ds.fb13, ds.wn18, ds.yago39, ds.dp50, ds.nations, ds.kinship, ds.umls], 
    kges=[kges.transE, kges.complEx, kges.distMult, kges.hoLE], 
    metrics=[brier_score, negative_log_loss, ks_error]
    )

In [7]:
# exp.load_trained_kges('../saved_models/')
exp.train_kges()

training TransE on FB13k ...


Average TransE Loss:   1.088411: 100%|██████████| 100/100 [19:01<00:00, 11.42s/epoch]


training TransE on WN11 ...


Average TransE Loss:   0.960336: 100%|██████████| 100/100 [08:26<00:00,  5.07s/epoch]


training TransE on YAGO39 ...


...

Average HolE Loss:   0.952201: 100%|██████████| 100/100 [00:15<00:00,  6.41epoch/s]


training HolE on Kinship ...


Average HolE Loss:   0.476121: 100%|██████████| 100/100 [00:46<00:00,  2.16epoch/s]


training HolE on UMLS ...


Average HolE Loss:   0.731894: 100%|██████████| 100/100 [00:37<00:00,  2.68epoch/s]


In [9]:
exp_res = exp.run_with_trained_kges()
# exp.save_trained_kges('../saved_models/new/')

training various calibrators for TransE on FB13k ...
True
training various calibrators for ComplEx on FB13k ...
True
training various calibrators for DistMult on FB13k ...
True
training various calibrators for HolE on FB13k ...
True
training various calibrators for TransE on WN11 ...
True
training various calibrators for ComplEx on WN11 ...
True
training various calibrators for DistMult on WN11 ...
True
training various calibrators for HolE on WN11 ...
True
training various calibrators for TransE on YAGO39 ...
True
training various calibrators for ComplEx on YAGO39 ...
False
training various calibrators for DistMult on YAGO39 ...
False
training various calibrators for HolE on YAGO39 ...
False
training various calibrators for TransE on DBpedia50 ...
True
training various calibrators for ComplEx on DBpedia50 ...
False
training various calibrators for DistMult on DBpedia50 ...
False
training various calibrators for HolE on DBpedia50 ...
True
training various calibrators for TransE on Nati

In [10]:
exp_res.to_frame()

ExpRes
dataset kge    cal                   metric                     
FB13k   TransE UncalCalibrator       brier_score        0.241895
                                     negative_log_loss  0.676001
                                     ks_error           0.099063
               PlattCalibrator       brier_score        0.211441
                                     negative_log_loss  0.614908
...                                                          ...
UMLS    HolE   BetaCalibrator        negative_log_loss  0.293783
                                     ks_error           0.013610
               TemperatureCalibrator brier_score        0.101561
                                     negative_log_loss  0.351584
                                     ks_error           0.084396

[504 rows x 1 columns]

In [ ]:
exp._train_cal_and_eval(exp.trained_kges['YAGO39']['ComplEx'], ds.yago39)

training various calibrators for ComplEx on YAGO39 ...


RuntimeError: all elements of input should be between 0 and 1